# LR Model Tuning

In this notebook, we will take the folds generated by the OU class in the previous notebook  and try to find the best set of parameters for our Logistic Regression Model to perform binary classification. 

In [1]:
import pandas as pd
import numpy as np

import sklearn
from sklearn import preprocessing

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

import pickle

import OU


%load_ext autoreload
%autoreload 2

In [2]:
save_dir = './data/'

directly read splits results calculated from "LR Model Tuning.ipynb"

In [3]:
splits = np.load(save_dir + "/splits.npy", allow_pickle=True)

In [4]:
multi_cv_df = pd.read_csv(save_dir + 'df.csv', index_col = 0)
multi_cv_labels = pd.read_csv(save_dir + 'labels.csv', index_col = 0)

# Gridsearch

We want to find the optimal hyperparameters for our LR by exploring all combinations of possible hyperparameter

In [5]:
params = [{
            'penalty' : ['l1','l2'], 
            'C'       : np.logspace(-3,3,7),
            'solver'  : ['newton-cg', 'lbfgs', 'liblinear'],
        }]

In [6]:
# Use all cores (n_jobs-1)
gridcv = GridSearchCV(LogisticRegression(), params, verbose = 1, cv = list(splits), n_jobs = -1, 
                    scoring = ['precision'], refit = False)

gridcv.fit(multi_cv_df, multi_cv_labels.values.ravel())

Fitting 399 folds for each of 42 candidates, totalling 16758 fits


/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision 

GridSearchCV(cv=[array([RangeIndex(start=0, stop=2000, step=1),
       RangeIndex(start=2000, stop=2100, step=1)], dtype=object),
                 array([RangeIndex(start=2100, stop=4100, step=1),
       RangeIndex(start=4100, stop=4200, step=1)], dtype=object),
                 array([RangeIndex(start=4200, stop=6200, step=1),
       RangeIndex(start=6200, stop=6300, step=1)], dtype=object),
                 array([RangeIndex(start=6300, s...
       RangeIndex(start=60800, stop=60900, step=1)], dtype=object),
                 array([RangeIndex(start=60900, stop=62900, step=1),
       RangeIndex(start=62900, stop=63000, step=1)], dtype=object), ...],
             estimator=LogisticRegression(), n_jobs=-1,
             param_grid=[{'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                          'penalty': ['l1', 'l2'],
                          'solver': ['newton-cg', 'lbfgs', 'liblinear']}],
             refit=False, scoring=['precision'], verbose=1)

In [7]:
gridcv_results = pd.DataFrame(gridcv.cv_results_)

In [8]:
best_param = gridcv_results[gridcv_results.rank_test_precision == 1]['params'].values[0]
print(f'Best Logistic Regression parameters after Grid Search is {best_param}.')

Best Logistic Regression parameters after Grid Search is {'C': 1000.0, 'penalty': 'l1', 'solver': 'liblinear'}.


In [9]:
# save the object to a file
with open(save_dir+'gridsearch_results_lr.pkl', 'wb') as f:
    pickle.dump(gridcv, f)